In [6]:
import pandas as pd
import numpy as np
import os
import torch
from tqdm import tqdm
from models import *
from dataSet.reader import *
from dataSet.transform import *
import os
import shutil

import os

import PIL
from PIL import Image
from PIL.ImageDraw import Draw

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from scipy.ndimage import affine_transform
from tensorflow.keras import backend as K

import pandas as pd
from tqdm import tqdm

In [7]:
import tensorflow as tf
tf.__version__

'2.1.0'

In [8]:

def train_collate(batch):

    batch_size = len(batch)
    images = [
    ]
    names = []
    for b in range(batch_size):
        if batch[b][0] is None:
            continue
        else:
            images.extend(batch[b][0])
            names.append(batch[b][1])
    images = torch.stack(images, 0)
    return images, names


def transform(image, mask):
    raw_iamge = cv2.resize(image, (512, 256))
    raw_mask = cv2.resize(mask, (512, 256))
    raw_mask = raw_mask[:, :, None]
    raw_iamge = np.concatenate([raw_iamge, raw_mask], 2)
    images = []

    image = raw_iamge.copy()
    image = np.transpose(image, (2, 0, 1))
    image = image.copy().astype(np.float)
    image = torch.from_numpy(image).div(255).float()
    images.append(image)

    image = raw_iamge.copy()
    image = np.fliplr(image)
    image = np.transpose(image, (2, 0, 1))
    image = image.copy().astype(np.float)
    image = torch.from_numpy(image).div(255).float()
    images.append(image)

    return images

In [11]:
checkPoint_start=0
fold_index=1
model_name='senet154'
DATA = './prediction'
names_test = os.listdir(DATA)

batch_size = len(names_test)

In [13]:
DATA

'./prediction'

In [14]:
names_test

['PM-WWA-201006024-149.jpg']

#### Create Mask

#### Create Bounding Box

In [15]:
model = load_model('cropping_2.model')

In [16]:

# Define useful constants
img_shape  =  (192, 384, 1) # (128, 128, 1) # 
# anisotropy = 2.15
anisotropy = 1

def center_transform(affine, input_shape):
    hi, wi = float(input_shape[0]), float(input_shape[1])
    ho, wo = float(img_shape[0]), float(img_shape[1])
    top, left, bottom, right = 0, 0, hi, wi
    if wi/hi/anisotropy < wo/ho: # input image too narrow, extend width
        w     = hi*wo/ho*anisotropy
        left  = (wi-w)/2
        right = left + w
    else: # input image too wide, extend height
        h      = wi*ho/wo/anisotropy
        top    = (hi-h)/2
        bottom = top + h
    center_matrix   = np.array([[1, 0, -ho/2], [0, 1, -wo/2], [0, 0, 1]])
    scale_matrix    = np.array([[(bottom - top)/ho, 0, 0], [0, (right - left)/wo, 0], [0, 0, 1]])
    decenter_matrix = np.array([[1, 0, hi/2], [0, 1, wi/2], [0, 0, 1]])
    return np.dot(np.dot(decenter_matrix, scale_matrix), np.dot(affine, center_matrix))

# Apply an affine transformation to an image represented as a numpy array.
def transform_img(x, affine):
    matrix   = affine[:2,:2]
    offset   = affine[:2,2]
    x        = np.moveaxis(x, -1, 0)
    channels = [affine_transform(channel, matrix, offset, output_shape=img_shape[:-1], order=1,
                                 mode='constant', cval=np.average(channel)) for channel in x]
    return np.moveaxis(np.stack(channels, axis=0), 0, -1)

def read_raw_image(p):
    return Image.open(p)

def read_for_validation(x):
    t  = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
    t  = center_transform(t, x.shape)
    x  = transform_img(x, t)
    x -= np.mean(x, keepdims=True)
    x /= np.std(x, keepdims=True) + K.epsilon()
    return x, t

def coord_transform(list, trans):
    result = []
    for x,y in list:
        y,x,_ = trans.dot([y,x,1]).astype(np.int)
        result.append((x,y))
    return result

def read_array(p):
    img = read_raw_image(p).convert('L')
    return img_to_array(img)

def make_bbox(p):
    raw = read_array(p)
    width, height = raw.shape[1], raw.shape[0]
    img,trans         = read_for_validation(raw)
    a                 = np.expand_dims(img, axis=0)
    x0, y0, x1, y1    = model.predict(a).squeeze()
    (u0, v0),(u1, v1) = coord_transform([(x0,y0),(x1,y1)], trans)
    bbox = [max(u0,0), max(v0,0), min(u1,width), min(v1,height)]
    if bbox[0] >= bbox[2] or bbox[1] >= bbox[3]:
        bbox = [0,0,width,height]
    return bbox

In [17]:
bbox_df = pd.DataFrame(columns=['Image','x0','y0','x1','y1']).set_index('Image')
for img in tqdm(names_test):
    bbox_df.loc[img] = make_bbox(os.path.join(DATA,img))

100%|██████████| 1/1 [00:00<00:00,  1.32it/s]


In [24]:
import os
if not os.path.exists('WC_input'):
    os.makedirs('WC_input')
bbox_df.to_csv("WC_input/bboxs.csv")

#### Create embeddings

In [22]:
dst_test = WhaleTestDataset(names_test, mode='test', transform=transform)
dataloader_test = DataLoader(dst_test, batch_size=batch_size, num_workers=0, collate_fn=train_collate)

loading bbox...


FileNotFoundError: [Errno 2] File ./input/label.csv does not exist: './input/label.csv'

In [ ]:
model.load_pretrain(os.path.join(checkPoint, '%08d_model.pth' % (checkPoint_start)),skip=[])
ckp = torch.load(os.path.join(checkPoint, '%08d_optimizer.pth' % (checkPoint_start)))
best_t = ckp['best_t']
print('best_t:', best_t)
labelstrs = []
allnames = []
with torch.no_grad():
    model.eval()
    for data in tqdm(dataloader_test):
        images, names = data
        images = images.cuda()
        _, _, outs = model(images)
        outs = torch.sigmoid(outs)
        outs_zero = (outs[::2, :5004] + outs[1::2, 5004:])/2
        outs = outs_zero
        for out, name in zip(outs, names):
            out = torch.cat([out, torch.ones(1).cuda()*best_t], 0)
            out = out.data.cpu().numpy()
            np.save(os.path.join(npy_dir, '{}.npy'.format(name)), out)
            top5 = out.argsort()[-5:][::-1]
            str_top5 = ''
            for t in top5:
                str_top5 += '{} '.format(id_label[t])
            str_top5 = str_top5[:-1]
            allnames.append(name)
            labelstrs.append(str_top5)
pd.DataFrame({'Image': allnames,'Id': labelstrs}).to_csv('test_{}_sub_fold{}.csv'.format(model_name, fold_index), index=None)

if __name__ == '__main__':
checkPoint_start = 41600
fold_index = 1
model_name = 'se_resnet50'
test(checkPoint_start, fold_index, model_name)

